In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import copy
import json
import os
import pickle
import typing
from collections import Counter, defaultdict
from copy import deepcopy
from pathlib import Path
from typing import Any, Dict, List, Set, Tuple

import numpy as np
import pandas as pd
import PIL.Image as pil_img
import seaborn as sns
import simdjson as json
from IPython.display import display
from PIL import Image
from pycocotools.coco import COCO
from tqdm.auto import tqdm

/home/gbiamby/mambaforge/envs/cocobetter/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and Inspect the Newly Saved RefCOCO Dataset

In [4]:
REFSEG_DIR = Path("/shared/gbiamby/data/refer_seg/")

In [6]:
VALID_SPLITS = {
    "R-refcoco": ["unc"],
    "R-refcoco+": ["unc"],
    "R-refcocog": ["umd"],
    "refclef": ["berkeley", "unc"],
    "refcoco": ["google"],
    "refcoco+": ["unc"],
    "refcocog": ["google", "umd"],
    # "fprefcoco_v002": ["berkeley"],
    # "fprefcoco+_v002": ["berkeley"],
    # "fprefcocog_v002": ["berkeley"],
    "fprefcoco_v002": ["berkeley_exclude_unified"],
    "fprefcoco+_v002": ["berkeley_exclude_unified"],
    "fprefcocog_v002": ["berkeley_exclude_unified"],
}


def build_refcoco(refseg_path: Path, dataset_name: str, split_by: str = None) -> COCO:
    assert dataset_name in VALID_SPLITS, dataset_name
    if split_by is None:
        split_by = VALID_SPLITS[dataset_name][0]
    else:
        assert split_by in VALID_SPLITS[dataset_name]
    coco = COCO(
        refseg_path / dataset_name / "instances.json",
        is_ref_dataset=True,
        dataset_name=dataset_name,
        split_by=split_by,
    )
    return coco


df_aggs = []
for ds_name in ["fprefcocog_v002"]:
    print("\n\n")
    print("=" * 220)
    print(f"Dataset: {ds_name}(berkeley)")
    coco = COCO(
        REFSEG_DIR,
        is_ref_dataset=True,
        dataset_name=ds_name,
        split_by="berkeley",
    )
    ## if you are using cocobetter you can output some stats by un-commenting these lines:
    # df_refcoco, df_refcoco_agg = coco.get_ref_stats()
    # df_aggs.append(df_refcoco_agg)


if len(df_aggs) > 0:
    df_aggs = pd.concat(df_aggs)
    display(df_aggs)




Dataset: fprefcocog_v002(berkeley)
Loading refs from '/shared/gbiamby/data/refer_seg/fprefcocog_v002/refs(berkeley).p'
Loaded 49822 refs
loading annotations into memory...
Done (t=17.77s)
creating index...
index created!


In [9]:
# Load the seem version to inspect it
seem_json_path = Path("/home/gbiamby/proj/SEEM/.xdecoder_data/coco/annotations/refcocog_umd_val.json")
seem_json = json.load(open(seem_json_path, "r"))
print("keys: ", seem_json.keys())
print(len(seem_json["images"]), len(seem_json["annotations"]))

keys:  dict_keys(['images', 'annotations'])
1300 2573


In [35]:
print(type(seem_json["images"]), type(seem_json["annotations"]))
print(type(seem_json["images"][0]), type(seem_json["annotations"][0]))
print(seem_json["images"][0].keys())
print(seem_json["annotations"][0].keys())
print(seem_json["annotations"][0]["sentences"][0])

<class 'list'> <class 'list'>
<class 'dict'> <class 'dict'>
dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])
dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id', 'split', 'sentences', 'file_name', 'ann_id', 'sent_ids', 'ref_id'])
{'tokens': ['a', 'bush', 'of', 'plant', 'behind', 'middle', 'woman'], 'raw': 'a bush of plant behind middle woman', 'sent_id': 61, 'sent': 'a bush of plant behind middle woman'}


In [49]:
def create_fp_ann(coco: COCO, ref: Ref):
    max_ann_id = max(coco.anns.keys())
    new_ann_id = max_ann_id + 1
    return {
        "segmentation": [[]],
        "area": 0.0,
        "iscrowd": 0,
        "image_id": ref["image_id"],
        "bbox": [0.0, 0.0, 0.0, 0.0],
        "category_id": "",
        "id": "",
    }


def convert_fpref(coco: COCO, output_dir: Path):
    print("Converting: ", coco.dataset_name)
    print("ref: ", coco.refs_data[0])
    print("image: ", list(coco.imgs.values())[0].keys())
    print("ann: ", list(coco.anns.values())[0].keys())
    print(len(coco.refs_data))
    splits = {r["split"] for r in coco.refs_data}
    print(splits)
    for split in splits:
        print("split: ", split)
        output_path = output_dir / f"{coco.dataset_name}_{split}.json"
        output_path.parent.mkdir(exist_ok=True, parents=True)
        data = {"images": [], "annotations": []}
        images, refs, annotations = {}, [], []
        ann_ids, ref_ids = {}, {}

        for i, ref in enumerate(coco.refs_data):
            if ref["split"] == split:
                refs.append(ref)
                images[ref["image_id"]] = coco.imgs[ref["image_id"]]
                ann_ids[ref["ann_id"]] = ref["ann_id"]
                ref_ids[ref["ref_id"]] = ref["ref_id"]
                new_ann: dict = deepcopy(coco.anns[ref["ann_id"]])
                sents = [
                    {
                        "tokens": s["tokens"],
                        "raw": s["raw"],
                        "sent_id": s["sent_id"],
                        "sent": s["sent"],
                    }
                    for s in ref["sentences"]
                ]
                new_ann.update(
                    {
                        "split": ref["split"],
                        "sentences": sents,
                        "file_name": ref["file_name"],
                        "ann_id": ref["ann_id"],
                        "sent_ids": ref["sent_ids"],
                        "ref_id": ref["ref_id"],
                    }
                )
                annotations.append(new_ann)
                if i == 0:
                    print(images[ref["image_id"]])
                    sents = new_ann["sentences"]
                    print(type(new_ann["sentences"]))
                    print(sents[0])

        print("Num imgs: ", len(images))
        print("Num refs: ", len(refs))
        print("Num ann_ids: ", len(ann_ids))
        print("Num ref_ids: ", len(ref_ids))
        print("New annotations: ", len(annotations))
        data = {"images": list(images.values()), "annotations": annotations}
        with open(output_path, "w") as f:
            json.dump(data, f)


for ds_name in ["fprefcoco_v002", "fprefcoco+_v002", "fprefcocog_v002"]:
    print("\n\n")
    print("=" * 220)
    print(f"Dataset: {ds_name}(berkeley)")
    coco = COCO(
        REFSEG_DIR,
        is_ref_dataset=True,
        dataset_name=ds_name,
        split_by="berkeley_exclude_unified",
    )
    convert_fpref(
        coco, Path("/home/gbiamby/proj/SEEM/.xdecoder_data/coco/fp_ref_annotations/")
    )




Dataset: fprefcoco_v002(berkeley)
Loading refs from '/shared/gbiamby/data/refer_seg/fprefcoco_v002/refs(berkeley_exclude_unified).p'
Loaded 50000 refs
loading annotations into memory...
Done (t=9.13s)
creating index...
index created!
Converting:  fprefcoco_v002
ref:  {'sent_ids': [13689, 13690, 13691], 'file_name': 'COCO_train2014_000000526754_0.jpg', 'ann_id': 592886, 'ref_id': 4808, 'image_id': 526754, 'split': 'val', 'sentences': [{'tokens': ['zebra', 'creature', 'front', 'and', 'center'], 'raw': 'zebra creature front and center', 'sent_id': 13689, 'sent': 'zebra creature front and center', 'spcy_WORD': ['zebra', 'creature', 'front', 'and', 'center'], 'spcy_DEP': ['compound', 'ROOT', 'advmod', 'cc', 'conj'], 'spcy_POS': ['NOUN', 'NOUN', 'ADJ', 'CCONJ', 'NOUN'], 'spcy_LEM': ['zebra', 'creature', 'front', 'and', 'center'], 'spcy_TAG': ['NN', 'NN', 'JJ', 'CC', 'NN'], 'spcy_IS_STOP': [False, False, True, True, False], 'spcy_ENTS': [], 'spcy_NOUN_CHUNKS': ['zebra creature'], 'is_false

In [48]:
def test_splits(coco: COCO):
    print("test_splits: ", coco.dataset_name)
    splits = {r["split"] for r in coco.refs_data}
    for split in splits:
        print("split: ", split)
        images, refs, annotations = {}, [], []
        ann_ids, ref_ids = {}, {}
        img_splits = set()
        for i, ref in enumerate(coco.refs_data):
            if ref["split"] == split:
                img = coco.imgs[ref["image_id"]]
                img_split = (
                    "trainval"
                    if "trainval" in img["file_name"]
                    else "train"
                    if "train" in img["file_name"]
                    else "test"
                    if "test" in img["file_name"]
                    else "val"
                    if "val" in img["file_name"]
                    else "UNKNOWN"
                )
                img_splits.add(img_split)
        print("split:", split, img_splits)


test_splits(coco)

test_splits:  fprefcocog_v002
split:  test
split: test {'train'}
split:  train
split: train {'train'}
split:  val
split: val {'train'}


The format is nearly identical to refcoco/refcocog. The difference is there are some additional properties in the sentences objects.

`coco->images->refs->sentences`

#### True sentences vs false premise:

These two methods are equivalent ways of detecting if a sentence is a false premise one:

`s["exist"] == False` and `s["is_false_premise"]`

Usually these sentences also have `sent_id = -1`, but that is not guaranteed so don't rely on it.


In [17]:
def display_ref(ref):
    print("ref has keys: ", ref.keys())
    print(f"ref has {len(ref['sentences'])} sentences")
    for s in ref["sentences"]:
        # print(s.keys())
        print(
            f"sent_id:{s['sent_id']}, is_FP:{s['is_false_premise']}, sent: '{s['sent']}'"
        )
        if s["is_false_premise"]:
            print("\tchange_type: ", s["change_type"])
            print(f"\tparent_sent_id: {s['gt_sent_id']}, parent_sent: '{s['gt_sent']}'")


def show_refexp_example(refcoco: COCO):
    ref = refcoco.refs[1000]
    display_ref(ref)


show_refexp_example(coco)

ref has keys:  dict_keys(['image_id', 'split', 'sentences', 'file_name', 'category_id', 'ann_id', 'sent_ids', 'ref_id'])
ref has 4 sentences
sent_id:21606, is_FP:False, sent: 'a large polar bear looking at a smaller polar bear'
sent_id:21607, is_FP:False, sent: 'white polar bear looking at another bear'
sent_id:-1, is_FP:True, sent: 'a large penguin looking at a smaller penguin'
	change_type:  
	parent_sent_id: 21606, parent_sent: 'a large polar bear looking at a smaller polar bear'
sent_id:-1, is_FP:True, sent: 'white penguin looking at another penguin'
	change_type:  
	parent_sent_id: 21607, parent_sent: 'white polar bear looking at another bear'



#### Change Type

Sentences in this dataset have a new property called `change_type`, indicating what kind of change was made to convert from the original ground truth sentence to the false premise one.

In [25]:
def show_change_type(coco: COCO, change_type: str):
    for ref_id, ref in coco.refs.items():
        for s in ref["sentences"]:
            if (
                "change_type" in s
                and s["change_type"]
                and s["change_type"] == "main_subject"
            ):
                print("Change_Type: ", s["change_type"])
                display_ref(ref)
                return


# show_change_type(coco, "NOT_MAIN_SUBJ")
show_change_type(coco, "main_subject")

Change_Type:  main_subject
ref has keys:  dict_keys(['image_id', 'split', 'sentences', 'file_name', 'category_id', 'ann_id', 'sent_ids', 'ref_id'])
ref has 4 sentences
sent_id:29, is_FP:False, sent: 'a truck number 14 on a snow bank'
sent_id:30, is_FP:False, sent: 'a truck with the number 14 painted on it'
sent_id:-1, is_FP:True, sent: 'a spaceship number 14 on a snow bank'
	change_type:  NOT_MAIN_SUBJ
	parent_sent_id: 29, parent_sent: 'a truck number 14 on a snow bank'
sent_id:-1, is_FP:True, sent: 'a spaceship with the number 14 painted on it'
	change_type:  main_subject
	parent_sent_id: 30, parent_sent: 'a truck with the number 14 painted on it'
